In [ ]:
X_init = processor.gestures[processor.CLEAN_SENSORS_FINAL].values
y_init = y

In [ ]:

# X_train = gestures[OMG_CH].values[:last_train_idx]
# y_train = y_cmd[:last_train_idx]

# X_test = gestures[OMG_CH].values[last_train_idx:]
# y_test = y_cmd[last_train_idx:]

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

scaler.fit(X_init)
X_scaled= scaler.transform(X_init)

In [ ]:
X_scaled[4:].shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, TimeDistributed,BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
time_steps = 50
X_train = X_scaled[4:]
y_train = y_init[4:]

features = 15
num_classes = 6

# X_train_reshaped = X_train
X_train_reshaped = X_train.reshape(-1, time_steps, features)
# X_test_reshaped = X_test.reshape(-1, time_steps, features)

y_train_one_hot = to_categorical(y_train)  # Формат (15320, 6)
# y_test_one_hot = to_categorical(y_test)    # Формат (3830, 

# y_train_gru = y_train_one_hot
y_train_gru = y_train_one_hot.reshape(y_train_one_hot.shape[0]//time_steps,time_steps, num_classes)
# y_test_gru = y_test_one_hot.reshape(y_test_one_hot.shape[0]//time_steps,time_steps,num_classes)

# # Вычисление весов классов
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights = dict(enumerate(class_weights))

# Создание модели с GRU
model = Sequential()
model.add(GRU(50, input_shape=(time_steps,features), activation='relu', return_sequences=True))  # 50 нейронов в слое

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(TimeDistributed(Dense(num_classes, activation='softmax')))
# model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
print(X_train_reshaped.shape)
print(y_train_gru.shape)


In [ ]:
model.fit(X_train_reshaped, y_train_gru, epochs=10, batch_size=32, validation_split = .2, callbacks=[early_stopping])

In [ ]:
predictions_train = model.predict(X_train_reshaped)
y_train_proba=np.reshape(predictions_train,(2473*50,6))
y_pred_train=np.argmax(y_train_proba,axis=1)
print(classification_report(y_train, y_pred_train, target_names=processor.GESTURES))